# **TMBD API Extraction**

_John Andrew Dixon_

---

##### **Imports**

In [24]:
import json, os, time
import pandas as pd
import tmdbsimple as tmdb
from tqdm.notebook import tqdm_notebook

##### **Helper Functions**

In [14]:
def get_movie_with_rating(movie_id):
    """Using the TMDB API, retrieves movie info and rating based on a given movie_id"""

    # Using the TMDB API, get movie based on "movie_id"
    movie = tmdb.Movies(movie_id)

    # Save movie info to a variable
    info = movie.info()
    # Save movie releases to a variable
    releases = movie.releases()

    # Loop through all releases for each country
    for country in releases["countries"]:
        # If the country is US, add the certification to the info dictionary
        if country["iso_3166_1"] == "US":
            info["certification"] = country["certification"]

    # Return the info dictionary
    return info

In [15]:
def write_json(new_data, filename):
    """Appends a list of records (new_data) to a json file (filename). 
    Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""

    with open(filename, "r+") as file:
        # Load existing data as a dictionary
        file_data = json.load(file)

        # Choose to extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
            file_data.append(new_data)

        # Set file's current position at offset
        file.seek(0)

        # Convert back to JSON
        json.dump(file_data, file)

##### **Data Load**

In [16]:
# Load the previous "basics" file from part 1
basics_df = pd.read_csv("Data/title_basics.csv.gz")
# Verify it loaded
basics_df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0035423,movie,Kate & Leopold,Kate & Leopold,0,2001.0,NaN,118,"Comedy,Fantasy,Romance"
1,tt0062336,movie,The Tango of the Widower and Its Distorting Mi...,El tango del viudo y su espejo deformante,0,2020.0,NaN,70,Drama
2,tt0069049,movie,The Other Side of the Wind,The Other Side of the Wind,0,2018.0,NaN,122,Drama
3,tt0088751,movie,The Naked Monster,The Naked Monster,0,2005.0,NaN,100,"Comedy,Horror,Sci-Fi"
4,tt0096056,movie,Crime and Punishment,Crime and Punishment,0,2002.0,NaN,126,Drama


---

## **Efficient TMBD API Extraction**

In [17]:
# Specify the folder where data will be saved
FOLDER = "Data/"
# Create the folder; if it exists, move on
os.makedirs(FOLDER, exist_ok=True)
# List folder's contents
os.listdir(FOLDER)

['title_akas.csv.gz', 'title_basics.csv.gz', 'title_ratings.csv.gz']

In [18]:
# Load API Key
with open("/Users/johna/.secret/tmdb_api.json", "r") as f:
    login = json.load(f)
login.keys()

dict_keys(['api-key'])

In [19]:
# Link API Key with API
tmdb.API_KEY = login["api-key"]

In [20]:
# Specify years to get
YEARS_TO_GET = [2000, 2001]
# Create empty list to hold potential errors
errors = []

In [30]:
# Loop through the years and display progress as progress bar
for YEAR in tqdm_notebook(YEARS_TO_GET, desc="YEARS", position=0):
    # Create a JSON file for the current YEAR in FOLDER
    JSON_FILE = f"{FOLDER}tmdb_api_results_{YEAR}.json"

    # Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    # If the file does not exists, create and populate it
    if not file_exists:
        with open(JSON_FILE, "w") as file:
            json.dump([{"imdb_id": 0}], file)

    # Filter the "basics_df" by the current YEAR
    year_df = basics_df.loc[basics_df["startYear"] == YEAR].copy()
    # Get the movie IDs from movies of the current year
    movie_ids = year_df["tconst"].copy()

    # Load in previous results
    previous_df = pd.read_json(JSON_FILE)
    # Filter out the movie ids to get based on what was previous gotten
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df["imdb_id"])]

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

8        tt0113026
9        tt0113092
11       tt0115937
12       tt0116391
13       tt0116628
           ...    
81728    tt8327752
82491    tt8553964
83609    tt8907070
83758    tt8954964
85236    tt9412476
Name: tconst, Length: 1444, dtype: object
0        tt0035423
10       tt0114447
17       tt0118589
18       tt0118652
23       tt0119004
           ...    
84208    tt9071078
84684    tt9212730
84719    tt9228234
85539    tt9555974
85599    tt9578462
Name: tconst, Length: 1568, dtype: object


In [21]:
# test = get_movie_with_rating("tt0848228")
# test

In [22]:
# test_ids = ["tt0848228", "tt0115937","tt0332280"]
# results = []

# for movie_id in test_ids:
    
#     try:
#         movie_info = get_movie_with_rating(movie_id)
#         results.append(movie_info)
        
#     except Exception as e:
#         errors.append((movie_id, e))
    
# pd.DataFrame(results)

In [23]:
# errors